In [ ]:
pip install numpy scipy soundfile pyroomacoustics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 47.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for pyroomacoustics: filename=pyroomacoustics-0.8.4-cp312-cp312-linux_x86_64.whl size=47010705 sha256=defcbc5858384d840aeee75812603c8cea788383090f35b4ad7ec0fb659a466c
  Stored in directory: /root/.cache/pip/wheels/71/2f/16/3cc6a78182189d4cde42c8f6658c1c8567cb4cc1146e61c9cb
Successfully built pyroomacoustics


In [ ]:
!pip install torch torchaudio

In [ ]:
import soundfile as sf

data1, sr1 = sf.read("/content/1272-128104-0009.flac")
data2, sr2 = sf.read("/content/1673-143396-0009.flac")

sf.write("source1.wav", data1, sr1)
sf.write("source2.wav", data2, sr2)


In [ ]:
import numpy as np
import pyroomacoustics as pra
import soundfile as sf
import scipy.signal
from numpy import load

# Parameters
mic_count = 8
mic_spacing = 0.08  # 8 cm
room_dim = [5, 5]  # 5m x 5m room
fs = 16000
source_angles = [30, 70]  # Degrees
source_distance = 1.5  # Distance from array center to sources
rir_duration = 0.4  # seconds
snr_db = 30

# Load Clean LibriSpeech Files
source1_audio, _ = sf.read('source1.wav')
source2_audio, _ = sf.read('source2.wav')

# Ensure both sources have same length for mixing
min_len = min(len(source1_audio), len(source2_audio))
source1_audio = source1_audio[:min_len]
source2_audio = source2_audio[:min_len]

#  Microphone Array Setup
# Center of mic array
array_center = np.array([room_dim[0]/2, room_dim[1]/2])

# Linear array along x-axis
mic_positions = np.zeros((2, mic_count))
mic_positions[0, :] = np.linspace(-mic_spacing*(mic_count-1)/2, mic_spacing*(mic_count-1)/2, mic_count)
mic_positions += array_center.reshape(2,1)

# Source Positions
source_positions = []
for angle_deg in source_angles:
    angle_rad = np.deg2rad(angle_deg)
    x = array_center[0] + source_distance * np.cos(angle_rad)
    y = array_center[1] + source_distance * np.sin(angle_rad)
    source_positions.append([x, y])

#  Room Setup
room = pra.ShoeBox(room_dim, fs=fs, max_order=10, absorption=0.4)

# Add sources
room.add_source(source_positions[0], signal=source1_audio)
room.add_source(source_positions[1], signal=source2_audio)

# Add mic array
mic_array_2d = mic_positions
room.add_microphone_array(mic_array_2d)

#  Simulate and Mix
room.simulate()

# Get mixed signals (shape: mic_count x samples)
mixed_signals = room.mic_array.signals

#  Save Mixed Signals
for i in range(mic_count):
    sf.write(f'mic_{i+1}.wav', mixed_signals[i], fs)


np.save("mixed_matrix.npy", mixed_signals)
print(" Saved as mixed_matrix.npy")
print(mixed_signals.shape)
matrix = np.load("mixed_matrix.npy")
data = load("mixed_matrix.npy")

 Saved as mixed_matrix.npy
(8, 265848)


In [ ]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.1/754.1 kB 29.8 MB/s eta 0:00:00


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,340 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ub

In [ ]:
!pip install -q jiwer torchaudio soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
import torchaudio
import soundfile as sf
import pyroomacoustics as pra
from speechbrain.inference.separation import SepformerSeparation as Separator
import whisper
from jiwer import wer

# ------------------ config ------------------
MIXED_NPY = "mixed_matrix.npy"     # shape: (M, T)
TARGET_SR = 8000                   # SepFormer expects 8 kHz
ARRAY_SR  = 16000                  # your array simulation fs
C = 343.0

# ------------------ helpers ------------------
def to_float(x):
    x = x.astype(np.float32)
    return x / (np.max(np.abs(x)) + 1e-9)

def resample_1d(x, orig, new):
    t = torch.tensor(x).float()
    if orig != new:
        t = torchaudio.functional.resample(t, orig_freq=orig, new_freq=new)
    return t.numpy()

def run_sepformer(mono, sr_in=16000, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    x8 = resample_1d(mono, sr_in, TARGET_SR)
    x8 = np.squeeze(x8)                  # ensure 1D
    wav = torch.tensor(x8).float().unsqueeze(0)  # [B,T]
    sep = Separator.from_hparams(
        source="speechbrain/sepformer-whamr",
        savedir="pretrained_models/sepformer-whamr",
        run_opts={"device": device}
    )
    with torch.no_grad():
        out = sep.separate_batch(wav.to(device))  # [B,T,2]
    s1 = out[0, :, 0].cpu().numpy()
    s2 = out[0, :, 1].cpu().numpy()
    return s1, s2, TARGET_SR

# ------------------ pipeline ------------------
# 1) load array mixture
assert os.path.exists(MIXED_NPY), f"{MIXED_NPY} not found"
mix_arr = np.load(MIXED_NPY)  # (M,T)
mix_arr = to_float(mix_arr)

# 2) pick center mic (baseline)
center_idx = mix_arr.shape[0] // 2
mono_center = mix_arr[center_idx]

# 3) run SepFormer
s1, s2, sr_sep = run_sepformer(mono_center, sr_in=ARRAY_SR)

# 4) save results
sf.write("est_source1.wav", s1, sr_sep, subtype="PCM_16")
sf.write("est_source2.wav", s2, sr_sep, subtype="PCM_16")
print(f" Separation complete → est_source1.wav / est_source2.wav @ {sr_sep} Hz")

/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoi

hyperparams.yaml: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-whamr/snapshots/9950fd8055b38679d2af5bef2366b5dba600ec66/hyperparams.yaml' -> '/content/pretrained_models/sepformer-whamr/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.parameter_transfer:Co

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-whamr/snapshots/9950fd8055b38679d2af5bef2366b5dba600ec66/masknet.ckpt' -> '/content/pretrained_models/sepformer-whamr/masknet.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["masknet"] = /content/pretrained_models/sepformer-whamr/masknet.ckpt
INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached


encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-whamr/snapshots/9950fd8055b38679d2af5bef2366b5dba600ec66/encoder.ckpt' -> '/content/pretrained_models/sepformer-whamr/encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["encoder"] = /content/pretrained_models/sepformer-whamr/encoder.ckpt
INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached


decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-whamr/snapshots/9950fd8055b38679d2af5bef2366b5dba600ec66/decoder.ckpt' -> '/content/pretrained_models/sepformer-whamr/decoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["decoder"] = /content/pretrained_models/sepformer-whamr/decoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: masknet, encoder, decoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): masknet -> /content/pretrained_models/sepformer-whamr/masknet.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): encoder -> /content/pretrained_models/sepformer-whamr/encoder.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): decoder -> /content/pretrained_models/sepformer-whamr/decoder.ckpt


 Separation complete → est_source1.wav / est_source2.wav @ 8000 Hz


In [ ]:
import numpy as np
from scipy.io import wavfile

mixed_matrix = np.load("mixed_matrix.npy")
mic1_waveform = mixed_matrix[0]
mic1_waveform = np.clip(mic1_waveform, -1.0, 1.0)
mic1_int16 = (mic1_waveform * 32767).astype(np.int16)
TARGET_SAMPLE_RATE = 16000
wavfile.write("mixed.wav", TARGET_SAMPLE_RATE, mic1_int16)
print(" Saved mic 1 waveform as mixed_mic1.wav")

 Saved mic 1 waveform as mixed_mic1.wav


In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from ipywidgets import VBox, Output
from IPython.display import display, Audio

# Load all files with actual sample rate
mix, sr_mix = sf.read("mixed.wav")
src1, sr1 = sf.read("est_source1.wav")
src2, sr2 = sf.read("est_source2.wav")

# Create time axes for plotting
t_mix = np.linspace(0, len(mix)/sr_mix, len(mix))
t1 = np.linspace(0, len(src1)/sr1, len(src1))
t2 = np.linspace(0, len(src2)/sr2, len(src2))

out_mix = Output()
out_s1 = Output()
out_s2 = Output()

with out_mix:
    print(" Mixed Signal")
    plt.figure(figsize=(10, 2))
    plt.plot(t_mix, mix, color='black')
    plt.title("Mixed Signal")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()
    display(Audio(mix, rate=sr_mix))

with out_s1:
    print(" Separated Source 1")
    plt.figure(figsize=(10, 2))
    plt.plot(t1, src1, color='green')
    plt.title("Separated Source 1")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()
    display(Audio(src1, rate=sr1))  # Correct sample rate

with out_s2:
    print(" Separated Source 2")
    plt.figure(figsize=(10, 2))
    plt.plot(t2, src2, color='blue')
    plt.title("Separated Source 2")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()
    display(Audio(src2, rate=sr2))  # Correct sample rate

display(VBox([out_mix, out_s1, out_s2]))


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
3

In [ ]:
# ------------------------ INSTALL DEPENDENCIES -------------------------
!pip install -q jiwer torchaudio soundfile

# ------------------------ IMPORT LIBRARIES -------------------------
import whisper
import os
import torchaudio
import soundfile as sf
from jiwer import wer
from IPython.display import Audio, display

# ------------------------ LOAD WHISPER MODEL -------------------------
model = whisper.load_model("small")

# ------------------------ FILE PATHS -------------------------
files = {
    "source1": "source1.wav",
    "source2": "source2.wav",
    "est_source1": "est_source1.wav",
    "est_source2": "est_source2.wav"
}

# ------------------------ TRANSCRIBE -------------------------
transcripts = {}
for label, path in files.items():
    if not os.path.exists(path):
        print(f" File not found: {path}")
        continue

    print(f"\n Transcribing {path}:")
    display(Audio(path))
    result = model.transcribe(path)
    transcripts[label] = result["text"].strip()
    print(f" {label} transcript: {transcripts[label]}")

# ------------------------ PERMUTATION-INVARIANT WER -------------------------
ref1, ref2 = transcripts["source1"], transcripts["source2"]
hyp1, hyp2 = transcripts["est_source1"], transcripts["est_source2"]

# Case 1: no swap
wer_case1 = wer(ref1.lower(), hyp1.lower()) + wer(ref2.lower(), hyp2.lower())
# Case 2: swapped
wer_case2 = wer(ref1.lower(), hyp2.lower()) + wer(ref2.lower(), hyp1.lower())

# ------------------------ ALIGN & SAVE -------------------------
TARGET_SR = 16000

if wer_case1 <= wer_case2:
    print("\n Best alignment: est_source1 ↔ source1, est_source2 ↔ source2")
    print(f"WER source1 vs est_source1: {wer(ref1.lower(), hyp1.lower()):.2%}")
    print(f"WER source2 vs est_source2: {wer(ref2.lower(), hyp2.lower()):.2%}")

    # Save aligned files
    audio1, sr1 = torchaudio.load("est_source1.wav")
    audio2, sr2 = torchaudio.load("est_source2.wav")
else:
    print("\n Best alignment: est_source1 ↔ source2, est_source2 ↔ source1 (swapped)")
    print(f"WER source1 vs est_source2: {wer(ref1.lower(), hyp2.lower()):.2%}")
    print(f"WER source2 vs est_source1: {wer(ref2.lower(), hyp1.lower()):.2%}")

    # Swap assignments
    audio1, sr1 = torchaudio.load("est_source2.wav")
    audio2, sr2 = torchaudio.load("est_source1.wav")

# Resample both to 16kHz and save
if sr1 != TARGET_SR:
    audio1 = torchaudio.functional.resample(audio1, sr1, TARGET_SR)
if sr2 != TARGET_SR:
    audio2 = torchaudio.functional.resample(audio2, sr2, TARGET_SR)

sf.write("est_source1_aligned.wav", audio1.squeeze().numpy(), TARGET_SR, subtype='PCM_16')
sf.write("est_source2_aligned.wav", audio2.squeeze().numpy(), TARGET_SR, subtype='PCM_16')

print(" Saved aligned outputs: est_source1_aligned.wav / est_source2_aligned.wav at 16kHz")

100%|███████████████████████████████████████| 461M/461M [00:13<00:00, 36.2MiB/s]



 Transcribing source1.wav:


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 source1 transcript: He laments most bitterly the divorce that has been made between decorative art and what we usually call pictures. Makes a customary appeal to the last judgment and reminds us that in the great days of art Michelangelo was the furnishing upholsterer.

 Transcribing source2.wav:


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 est_source1 transcript: She laments most bitterly the divorce that has been made between decorative art and what we usually call pictures. It makes a customary appeal for the last judgment and reminds us that in the great stages of art Michelangelo was the...

 Transcribing est_source2.wav:


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 est_source2 transcript: He first appeared on the banks of the Jordan in the form of perfect manhood, but it was a form only and not a substance. A human figure created by the hand of a misfit to imitate the faculties and actions of a man and to impose a perpetual illusion on the senses of his friends and enemies.

 Best alignment: est_source1 ↔ source1, est_source2 ↔ source2
WER source1 vs est_source1: 16.67%
WER source2 vs est_source2: 8.77%
 Saved aligned outputs: est_source1_aligned.wav / est_source2_aligned.wav @16kHz


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

1. WER source1 vs est_source2: 16.67%

2. WER source2 vs est_source1: 8.77%